### Generate the big co-occurrence matrix

1. load the dictionary and add numeric indices
2. Load the song data into dataframe
3. Convert the word keys to words
4. (skip) replace word stems with full words
5. For each row, sum the counts in each column
6. This is your co-occurence matrix

In [1]:
import pandas as pd

In [3]:
# reading csv file  
matched = pd.read_csv(r"../Py code/words_songs_matched.csv") 

In [4]:
matched.head()

,word,count,track_id,song_id,artist_id,title,artist_name,duration,year,artist_hotttnesss
0,i,6,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
1,the,4,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
2,you,2,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
3,to,2,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
4,and,5,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606


In [5]:
matched.drop(columns=['track_id', 'song_id', 'artist_id', 'duration', 'artist_hotttnesss'], inplace=True)

In [6]:
matched.head()

,word,count,title,artist_name,year
0,i,6,A Poor Recipe For Civic Cohesion,Western Addiction,2005
1,the,4,A Poor Recipe For Civic Cohesion,Western Addiction,2005
2,you,2,A Poor Recipe For Civic Cohesion,Western Addiction,2005
3,to,2,A Poor Recipe For Civic Cohesion,Western Addiction,2005
4,and,5,A Poor Recipe For Civic Cohesion,Western Addiction,2005


In [7]:
matched.to_csv(r'../Py code/words_songs_matched_lite.csv', index = False)

### Now we have the trimmed table, time to reswizzle the axes

In [12]:
# find the number of unique words, it should be 5000
uniques = matched.word.unique()

In [17]:
len(uniques)

5000

### Stop using the above code and start from here decreases memory usage since only loading 900MB csv instead of 2GB

In [22]:
# reading csv file  
matched = pd.read_csv(r"../Py code/words_songs_matched_lite.csv") 

In [4]:
matched.head()

,word,count,title,artist_name,year
0,i,6,A Poor Recipe For Civic Cohesion,Western Addiction,2005
1,the,4,A Poor Recipe For Civic Cohesion,Western Addiction,2005
2,you,2,A Poor Recipe For Civic Cohesion,Western Addiction,2005
3,to,2,A Poor Recipe For Civic Cohesion,Western Addiction,2005
4,and,5,A Poor Recipe For Civic Cohesion,Western Addiction,2005


In [25]:
matched.shape

(19045332, 5)

In [11]:
uniquewords = matched.word.unique()

In [12]:
# never do this to the full matched table. If an extra 5 columns is 1GB of file size, adding 5000 cols is stupid

# for w in uniquewords:
#     matched["w"] = ""

In [14]:
# this inplace was a bad idea since you need the full table to recover the word counts
matched.drop_duplicates(subset='title', inplace=True)

In [15]:
matched.head()

,word,count,title,artist_name,year,w
0,i,6,A Poor Recipe For Civic Cohesion,Western Addiction,2005,
68,i,10,Soul Deep,The Box Tops,1969,
142,i,28,It's About Time,Jamie Cullum,0,
262,i,5,Something Girls,Adam Ant,1982,
321,i,4,Burn My Body (Album Version),Broken Spindles,0,


In [16]:
liteframe = matched.drop(columns=['word', 'count', 'w'])

In [20]:
liteframe.head()

,title,artist_name,year
0,A Poor Recipe For Civic Cohesion,Western Addiction,2005
68,Soul Deep,The Box Tops,1969
142,It's About Time,Jamie Cullum,0
262,Something Girls,Adam Ant,1982
321,Burn My Body (Album Version),Broken Spindles,0


In [21]:
liteframe.shape

(190525, 3)

In [23]:
liteframe.reset_index(drop=True)

,title,artist_name,year
0,A Poor Recipe For Civic Cohesion,Western Addiction,2005
1,Soul Deep,The Box Tops,1969
2,It's About Time,Jamie Cullum,0
3,Something Girls,Adam Ant,1982
4,Burn My Body (Album Version),Broken Spindles,0
...,...,...,...
190520,Dog Food (Live),Kids Like Us,0
190521,Solo Dolo (nightmare),Kid Cudi,2009
190522,Operator's Manual,The Buzzcocks,0
190523,Toca Me,Fragma,1999


So here's a problem:  
The matched table is roughly 900MB and is 19M * 5 = 95M cells  
The liteframe table is 190K rows times 5000 columns = 950M cells >> 9GB  
How is this an improvement??  

And this is without even the word co-occurence

In [26]:
liteframe.groupby('year')['title'].nunique()

year
0       51592
1924        1
1925        1
1926        1
1927        8
        ...  
2006     9758
2007    10044
2008     8202
2009     6309
2010     1270
Name: title, Length: 87, dtype: int64